In [1]:
%reload_ext autoreload
%autoreload 2

# Imports

In [2]:
from kret_notebook import *  # NOTE import first
from kret_matplotlib.mpl_nb_imports import *
from kret_np_pd.np_pd_nb_imports import *
from kret_sklearn.sklearn_nb_imports import *
from kret_torch_utils.torch_nb_imports import *
from kret_lightning.lightning_nb_imports import *
from kret_tqdm.tqdm_nb_imports import *
from kret_type_hints.types_nb_imports import *
from kret_utils.utils_nb_imports import *

# from kret_wandb.wandb_nb_imports import *  # NOTE this is slow to import

Loaded environment variables from /Users/Akseldkw/coding/kretsinger/.env
[kret_matplotlib.mpl_nb_imports] Imported kret_matplotlib.mpl_nb_imports in 2.2127 seconds
[kret_np_pd.np_pd_nb_imports] Imported kret_np_pd.np_pd_nb_imports in 1.1358 seconds
[kret_sklearn.sklearn_nb_imports] Imported kret_sklearn.sklearn_nb_imports in 0.5304 seconds
[kret_torch_utils.torch_nb_imports] Imported kret_torch_utils.torch_nb_imports in 2.9108 seconds
[kret_lightning.lightning_nb_imports] Imported kret_lightning.lightning_nb_imports in 0.1362 seconds
[kret_tqdm.tqdm_nb_imports] Imported kret_tqdm.tqdm_nb_imports in 0.0000 seconds
[kret_type_hints.types_nb_imports] Imported kret_type_hints.types_nb_imports in 0.0024 seconds
[kret_utils.utils_nb_imports] Imported kret_utils.utils_nb_imports in 0.0007 seconds


In [3]:
from lightning.pytorch.loggers import CSVLogger

# Load Data

In [4]:
from mnist_data import MNISTDataModule

mnist_data_module = MNISTDataModule(DATA_DIR / "MNIST")

# Implementation

In [5]:
from kret_lightning.base_lightning_nn import HPasKwargs


class Kret_AutoEncoder(BaseLightningNN):
    _criterion: nn.Module = nn.MSELoss()

    def __init__(self, embedding_dim: tuple[int, int], **kwargs: t.Unpack[HPasKwargs]):
        super().__init__(**kwargs)
        # define any number of nn.Modules (or use your current ones)
        self.encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(*embedding_dim))
        self.decoder = nn.Sequential(nn.Linear(embedding_dim[1], embedding_dim[0]), nn.ReLU(), nn.Linear(64, 28 * 28))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # print(f"Input shape: {x.shape}")
        x = x.view(x.size(0), -1)  # flatten
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    # endregion
    # region Training / Validation Steps
    def training_step(self, batch: tuple[torch.Tensor, torch.Tensor], batch_idx: int) -> torch.Tensor:
        """

        loss = ...
        return loss
        """
        x, _ = batch
        x = x.view(x.size(0), -1)
        outputs = self(x)
        loss = self.get_loss(outputs, x)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch: tuple[torch.Tensor, torch.Tensor], batch_idx: int) -> None:
        """
        val_loss = ...
        self.log('val_loss', val_loss)
        """
        x, _ = batch
        x = x.view(x.size(0), -1)
        outputs = self(x)
        val_loss = self.get_loss(outputs, x)
        self.log("val_loss", val_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

    # endregion

In [6]:
base = Kret_AutoEncoder((64, 3))
auto_enc = base

In [9]:
base.save_load_logging_dict

{'save_dir': PosixPath('/Users/Akseldkw/coding/data_kretsinger/lightning_logs'),
 'name': 'Kret_AutoEncoder__lr=0.001--gamma=0.5--stepsize=12--embedding_dim=(64,3)',
 'version': 'v_000'}

In [10]:
logger = CSVLogger(**base.save_load_logging_dict)

In [12]:
static_args = TrainerStaticDefaults.TRAINER_QUICK_ITER
dynamic_args = TrainerDynamicDefaults.trainer_dynamic_defaults(auto_enc, mnist_data_module)
trainer_args = static_args | dynamic_args
trainer = L.Trainer(**trainer_args)

Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


In [13]:
trainer.fit(model=auto_enc, datamodule=mnist_data_module)

┏━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name    ┃ Type       ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ encoder │ Sequential │ 50.4 K │ train │     0 │
│ 1 │ decoder │ Sequential │ 51.2 K │ train │     0 │
└───┴─────────┴────────────┴────────┴───────┴───────┘

Trainable params: 101 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 101 K                                                                                                
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 8                                                                                           
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

/Users/Akseldkw/micromamba/envs/kret_312/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Output()

/Users/Akseldkw/micromamba/envs/kret_312/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:434: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


`Trainer.fit` stopped: `max_steps=1` reached.


# Sandbox